In [3]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/46.5 MB ? eta -:--:--
   ------ --------------------------------- 7.3/46.5 MB 39.9 MB/s eta 0:00:01
   ----------- ---------------------------- 12.8/46.5 MB 31.7 MB/s eta 0:00:02
   ------------ --------------------------- 14.9/46.5 MB 24.2 MB/s eta 0:00:02
   ----------------- ---------------------- 19.9/46.5 MB 24.4 MB/s eta 0:00:02
   --------------------- ------------------ 25.2/46.5 MB 24.6 MB/s eta 0:00:01
   ----------------------- ---------------- 27.0/46.5 MB 22.3 MB/s eta 0:00:01
   ---------------------------- ----------- 33.0/46.5 MB 23.1 MB/s eta 0:00:01
   ------------------------------- -------- 36.7/46.5 MB 23.8 MB/s eta 0:00:01
   -------------------------------- ------- 37.7/46.5 MB 20.6 MB/s eta 0:00:01
   ------------------------------------- -- 43.5/46.5 MB 21.3 MB/s eta 0:00:01
   ---------------------------------------- 46.5/46.5 MB 20.9 M

In [ ]:
load_dotenv()

endpoint = os.getenv("ENDPOINT_URL")
deployment = os.getenv("DEPLOYMENT_NAME")
search_endpoint = os.getenv("SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_KEY")
search_index = os.getenv("SEARCH_INDEX_NAME")
api_key = os.getenv("AZURE_OPENAI_AI_KEY")

print(endpoint, deployment, search_endpoint, search_key, search_index, api_key)

NameError: name 'load_dotenv' is not defined

In [ ]:
import os  
import requests
import json
import base64
from openai import AzureOpenAI  
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file
def request_gpt(user_text):
    endpoint = os.getenv("ENDPOINT_URL")
    deployment = os.getenv("DEPLOYMENT_NAME")
    search_endpoint = os.getenv("SEARCH_ENDPOINT")
    search_key = os.getenv("SEARCH_KEY")
    search_index = os.getenv("SEARCH_INDEX_NAME")
    api_key = os.getenv("AZURE_OPENAI_AI_KEY")

    api_version = "2024-05-01-preview"
    url = f"{endpoint}openai/deployments/{deployment}/chat/completions?api-version={api_version}"
    
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    body = {
        "messages": [
            {"role": "system", "content": "너는 부동산 및 청년 주택 매매·임대 관련 공문을 기반으로 정보를 제공하는 AI야. 가능한 한 한 개의 문서만 참조해서 실용적이고 날짜 중심의 정보를 표로 정리해줘."},
            {"role": "user", "content": user_text}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": 1000,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": "default",
                    "query_type": "vector_semantic_hybrid",
                    "in_scope": True,
                    "top_n_documents": 1,
                    "strictness": 3,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    },
                    "embedding_dependency": {
                        "type": "integrated"
                    }
                }
            }
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))
    
    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        return f"요청 실패: {response.status_code} - {response.text}"

def main():
    print("Azure OpenAI 부동산 상담 AI (종료하려면 'exit' 또는 '나가기' 입력)")
    print("-----------------------------------------------------")
    
    while True:
        user_input = input("질문: ")
        if user_input.lower() in ['exit', '나가기']:
            print("프로그램을 종료합니다.")
            break
            
        print("\n처리 중...\n")
        response = request_gpt(user_input)
        print(f'질문: {user_input}')
        print('-' * 50)
        print(f'답변: {response}')
        print('*' * 50)
        print()

if __name__ == "__main__":
    main()

Azure OpenAI 부동산 상담 AI (종료하려면 'exit' 또는 '나가기' 입력)
-----------------------------------------------------

처리 중...

질문: 안녕
--------------------------------------------------
답변: 요청 실패: 401 - {"error":{"code":"401","message":"Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource."}}
**************************************************

